# First Activity

In [1]:
# importing libraries needed
import pandas as pd 
import requests                             #module helps to download a web page 
!pip install bs4
from bs4 import BeautifulSoup               #module helps in web scrapping
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
data = requests.get(url).text
# table_contents=['Postalcode', 'Borough', 'Neighborhood']
table_contents=[]
soup = BeautifulSoup(data, 'html5lib')
table = soup.find('table')

for row in table.findAll('td'):
    #cell = {'Postalcode','Borough','Neighborhood'}
    cell={}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postalcode'] = row.p.text[:3]                #prints only first 3 elements in cell
        cell['Borough'] = (row.span.text).split('(')[0]
        
        #cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')',' ')).strip(' ')
        if ( '(' in row.span.text):
            cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /', ',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

#print (table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})


In [2]:
df.shape

(103, 3)

# Second Activity

In [3]:
#converting Geospatial Data to a dataframe
df_latlng = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

#Adding Latitude and Longitude Columns to Original dataframe
Latitude = df_latlng['Latitude']
Longitude = df_latlng['Longitude']
df = df.join(Latitude).join(Longitude)

#Viewing new dataset
df.head(12)

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Queen's Park,Ontario Provincial Government,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills North,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


# Third Activity

In [4]:
import json #library for json files
import numpy as np #library for numerical calculations 

!conda install -c conda-forge geopy #Foursquare API

from geopy.geocoders import Nominatim #convert address into latitude and longitude

from pandas.io.json import json_normalize #transfer JSON file into pandas dataframe

#matplotlib 
import matplotlib.cm as cm
import matplotlib.colors as colors

#k-means for clustering
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0
import folium  #map rendering library

print("Libraries loaded.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.8-main

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2021.5.30          |   py38h578d9bd_0         141 KB  conda-forge
    geographiclib-1.52         |     pyhd8ed1ab_0          35 KB  conda-forge
    geopy-2.2.0                |     pyhd8ed1ab_0          67 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.8             |           2_cp38           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.4 MB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.52-pyhd8ed1ab_0
  geopy              conda-forge/noarch::geopy-2.2.0-pyhd8ed1ab_0
  pyth

### Coordinates of Toronto

In [5]:
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.6534817, -79.3839347.


#### Number of Boroughs and Neighborhoods

In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


#### Create a map of Toronto showing neighborhoods

In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format('Neighborhood', 'Borough')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Exploring Borough names containing Toronto 

In [8]:
df_Toronto = df[df['Borough'].astype(str).str.contains('Toronto')].reset_index(drop=True)
df_Toronto.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,43.786947,-79.385975
4,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714


### Visualize df_Toronto

In [9]:
# create map of df_Toronto using latitude and longitude values
map_df_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_Toronto['Latitude'], df_Toronto['Longitude'], df_Toronto['Borough'], df_Toronto['Neighborhood']):
    label = '{}, {}'.format('Neighborhood', 'Borough')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_df_toronto)  
    
map_df_toronto

### Define Foursquare Credentials

In [10]:
CLIENT_ID = 'AZOP1JROPQZVYUIZGPTHGBVXRCBCDR2S01HFEWMKMVZWJETN' # your Foursquare ID
CLIENT_SECRET = '4NQYR5JXIPAEXSLE15WNDGIOOKCX1K4DOI1UCYQNYSMS2DRX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: AZOP1JROPQZVYUIZGPTHGBVXRCBCDR2S01HFEWMKMVZWJETN
CLIENT_SECRET:4NQYR5JXIPAEXSLE15WNDGIOOKCX1K4DOI1UCYQNYSMS2DRX


### Explore the first neighborhood

In [11]:
df_Toronto.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [12]:
neighborhood_latitude = df_Toronto.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df_Toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = df_Toronto.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


### To get the top 100 Venues within a 500meter radius of Frist Neighborhood

In [13]:
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius
 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=AZOP1JROPQZVYUIZGPTHGBVXRCBCDR2S01HFEWMKMVZWJETN&client_secret=4NQYR5JXIPAEXSLE15WNDGIOOKCX1K4DOI1UCYQNYSMS2DRX&v=20180605&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [14]:
results= requests.get(url).json()


# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# clean json file and structure to a pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()




,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


In [15]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


### Create a function to repeat the same process to all the neighborhoods

In [16]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

## Explore Nearby_Venues in Borough containing Toronto dataframe

In [17]:
df_Toronto_venues = getNearbyVenues(names=df_Toronto['Neighborhood'],
                                   latitudes=df_Toronto['Latitude'],
                                   longitudes=df_Toronto['Longitude']
                                  )


Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

In [18]:
#Print the number of venues
print('{} venues were returned by Foursquare.'.format(df_Toronto_venues.shape[0]))

752 venues were returned by Foursquare.


In [19]:
#Evaluate the size of the new dataframe
print(df_Toronto_venues.shape)
df_Toronto_venues.head()


(752, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [20]:
#Classify by neighborhood
df_Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Brockton, Parkdale Village, Exhibition Place",36,36,36,36,36,36
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,6,6,6,6,6,6
Christie,3,3,3,3,3,3
Church and Wellesley,9,9,9,9,9,9
"Commerce Court, Victoria Hotel",3,3,3,3,3,3
Davisville,4,4,4,4,4,4
Davisville North,67,67,67,67,67,67
"Dufferin, Dovercourt Village",5,5,5,5,5,5


In [21]:
print('There are {} uniques categories.'.format(len(df_Toronto_venues['Venue Category'].unique())))

There are 200 uniques categories.


In [22]:
# one hot encoding
df_Toronto_onehot = pd.get_dummies(df_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
df_Toronto_onehot['Neighborhood'] = df_Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [df_Toronto_onehot.columns[-1]] + list(df_Toronto_onehot.columns[:-1])
df_Toronto_onehot = df_Toronto_onehot[fixed_columns]

df_Toronto_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,Art Gallery,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
df_Toronto_grouped = df_Toronto_onehot.groupby('Neighborhood').mean().reset_index()
df_Toronto_grouped

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Arepa Restaurant,...,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,"Brockton, Parkdale Village, Exhibition Place",0.027778,0.000000,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.062500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
2,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
3,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
4,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
5,"Commerce Court, Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
6,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
7,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.014925,...,0.014925,0.00000,0.000000,0.000000,0.00,0.044776,0.059701,0.014925,0.000000,0.00
8,"Dufferin, Dovercourt Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00
9,Enclave of M4L,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00


In [24]:
df_Toronto_grouped.shape

(37, 200)

### Print each neighborhood along with the top 5 most common venues

In [25]:
num_top_venues = 5

for hood in df_Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = df_Toronto_grouped[df_Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Brockton, Parkdale Village, Exhibition Place----
                 venue  freq
0          Coffee Shop  0.08
1              Brewery  0.06
2                 Café  0.06
3               Bakery  0.06
4  American Restaurant  0.06


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0    Light Rail Station  0.12
1           Yoga Studio  0.06
2            Skate Park  0.06
3         Garden Center  0.06
4  Fast Food Restaurant  0.06


----Central Bay Street----
         venue  freq
0  Pizza Place  0.17
1      Butcher  0.17
2     Pharmacy  0.17
3  Coffee Shop  0.17
4  Supermarket  0.17


----Christie----
                  venue  freq
0  Fast Food Restaurant  0.33
1                  Park  0.33
2     Food & Drink Shop  0.33
3         Movie Theater  0.00
4        Medical Center  0.00


----Church and Wellesley----
                       venue  freq
0                Pizza Place  0.22
1                Coff

### Creating a dataframe with the top 10 venues

Define a function to sort venues in descending order and create the new dataframe to display the top 10 venues for each neighborhood.

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#create dataframe
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_Toronto_grouped['Neighborhood']

for ind in np.arange(df_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Brockton, Parkdale Village, Exhibition Place",Coffee Shop,Brewery,Café,Bakery,American Restaurant,Gastropub,Stationery Store,Bookstore,Diner,Middle Eastern Restaurant
1,"CN Tower, King and Spadina, Railway Lands, Har...",Light Rail Station,Yoga Studio,Skate Park,Garden Center,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Comic Shop,Restaurant
2,Central Bay Street,Pizza Place,Butcher,Pharmacy,Coffee Shop,Supermarket,Discount Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
3,Christie,Fast Food Restaurant,Park,Food & Drink Shop,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
4,Church and Wellesley,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Intersection,Chinese Restaurant,Playground,Discount Store,Optical Shop,Mexican Restaurant


## Cluster Neighborhoods

In [27]:
# set number of clusters
kclusters = 5

df_Toronto_grouped_clustering = df_Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 0, 4, 0, 0, 4, 1, 0], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_) 

Toronto_merged = df_Toronto

# merge df_Toronto_grouped with df_Toronto to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood', how= 'right')

Toronto_merged.head() 
Toronto_merged

,Postalcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.659526,-79.340923,4,Coffee Shop,Brewery,Café,Bakery,American Restaurant,Gastropub,Stationery Store,Bookstore,Diner,Middle Eastern Restaurant
32,M5V,Downtown Toronto,"CN Tower, King and Spadina, Railway Lands, Har...",43.662744,-79.321558,4,Light Rail Station,Yoga Studio,Skate Park,Garden Center,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Comic Shop,Restaurant
5,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259,4,Pizza Place,Butcher,Pharmacy,Coffee Shop,Supermarket,Discount Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
6,M6G,Downtown Toronto,Christie,43.753259,-79.329656,0,Fast Food Restaurant,Park,Food & Drink Shop,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
37,M4Y,Downtown Toronto,Church and Wellesley,43.696319,-79.532242,4,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Intersection,Chinese Restaurant,Playground,Discount Store,Optical Shop,Mexican Restaurant
16,M5L,Downtown Toronto,"Commerce Court, Victoria Hotel",43.689574,-79.383160,0,Restaurant,Tennis Court,Park,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
26,M4S,Central Toronto,Davisville,43.713756,-79.490074,0,Construction & Landscaping,Park,Basketball Court,Bakery,Yoga Studio,Movie Theater,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
20,M4P,Central Toronto,Davisville North,43.653206,-79.400049,4,Café,Vietnamese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Park,Cocktail Bar,Dumpling Restaurant,Burger Joint
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944,1,Grocery Store,Park,Shopping Mall,Bank,Monument / Landmark,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
38,M7Y,East Toronto Business,Enclave of M4L,43.688905,-79.554724,0,Park,Mobile Phone Shop,Sandwich Place,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


# Visualize Clusters

In [30]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine Clusters

## Cluster 1

In [31]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Downtown Toronto,0,Fast Food Restaurant,Park,Food & Drink Shop,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
16,Downtown Toronto,0,Restaurant,Tennis Court,Park,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
26,Central Toronto,0,Construction & Landscaping,Park,Basketball Court,Bakery,Yoga Studio,Movie Theater,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
38,East Toronto Business,0,Park,Mobile Phone Shop,Sandwich Place,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
10,Downtown Toronto,0,Intersection,Athletics & Sports,Spa,Beer Store,Bus Stop,Skating Rink,Curling Ice,Park,Other Great Outdoors,Molecular Gastronomy Restaurant
7,Downtown Toronto,0,Construction & Landscaping,Airport,Park,Bus Stop,Museum,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
24,Central Toronto,0,Park,Women's Store,Pool,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop


## Cluster 2

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,West Toronto,1,Grocery Store,Park,Shopping Mall,Bank,Monument / Landmark,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


## Cluster 3

In [33]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,Central Toronto,2,Field,Trail,Hockey Arena,Movie Theater,Lounge,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


## Cluster 4

In [34]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
36,Downtown Toronto,3,Baseball Field,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant
33,Downtown Toronto,3,Construction & Landscaping,Pool,Baseball Field,Museum,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant


## Cluster 5

In [35]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,West Toronto,4,Coffee Shop,Brewery,Café,Bakery,American Restaurant,Gastropub,Stationery Store,Bookstore,Diner,Middle Eastern Restaurant
32,Downtown Toronto,4,Light Rail Station,Yoga Studio,Skate Park,Garden Center,Fast Food Restaurant,Farmers Market,Park,Pizza Place,Comic Shop,Restaurant
5,Downtown Toronto,4,Pizza Place,Butcher,Pharmacy,Coffee Shop,Supermarket,Discount Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
37,Downtown Toronto,4,Pizza Place,Coffee Shop,Middle Eastern Restaurant,Sandwich Place,Intersection,Chinese Restaurant,Playground,Discount Store,Optical Shop,Mexican Restaurant
20,Central Toronto,4,Café,Vietnamese Restaurant,Bakery,Vegetarian / Vegan Restaurant,Coffee Shop,Bar,Park,Cocktail Bar,Dumpling Restaurant,Burger Joint
34,Downtown Toronto Stn A,4,Gym,Thrift / Vintage Store,Burrito Place,Fast Food Restaurant,Sandwich Place,Supplement Shop,Bakery,Discount Store,Tanning Salon,Burger Joint
21,Central Toronto,4,Airport Service,Airport Terminal,Coffee Shop,Airport,Bar,Sculpture Garden,Rental Car Location,Boat or Ferry,Plane,Harbor / Marina
1,Downtown Toronto,4,Skating Rink,Farm,College Stadium,General Entertainment,Café,Museum,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop
22,West Toronto,4,Coffee Shop,Italian Restaurant,Hotel,Gym,Cocktail Bar,Japanese Restaurant,Seafood Restaurant,Café,Restaurant,Bakery
15,East Toronto,4,Dessert Shop,Sandwich Place,Sushi Restaurant,Gym,Pizza Place,Coffee Shop,Café,Thai Restaurant,Italian Restaurant,Gourmet Shop
